In [1]:
import gzip
import math
import matplotlib.pyplot as plt
import numpy
import random
import sklearn
import string
from collections import defaultdict
from gensim.models import Word2Vec
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

# load and process data

In [2]:
import gzip
import json
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [27]:
data = parse("./Video_Games.json.gz")

In [28]:
import random
data = list(data)
random.seed(0)
random.shuffle(data)

In [29]:
traindata = data[:50000]
validdata = data[50000:75000]
testdata = data[75000:100000]

In [30]:
traindata[-1]

{'overall': 1.0,
 'verified': True,
 'reviewTime': '12 14, 2016',
 'reviewerID': 'A3VK4801K2E8MG',
 'asin': 'B000WMEEBC',
 'style': {'Platform:': ' PlayStation 3'},
 'reviewerName': 'King Zulu',
 'reviewText': 'Terrible, cannot compete with the PC version...',
 'summary': 'One Star',
 'unixReviewTime': 1481673600}

In [31]:
traindata[1].keys()

dict_keys(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'])

In [32]:
from collections import defaultdict
userset = set()
itemset = set()
averageRating = 0
ratingCount = defaultdict(int)
acc = 0
counter = 0


for d in traindata:
    userset.add(d['reviewerID'])
    itemset.add(d['asin'])
    acc += float(d['overall'])
    counter+=1
    ratingCount[d['overall']] += 1
averageRating = acc/counter

## TF-IDF

## TF code

In [33]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10713\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\10713\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
traindata = data[:75000]
testdata = data[75000:100000]

In [41]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [42]:
train_X , test_X = [] , [] 

for i in range(0, len(traindata)):
    words = ""
    if('reviewText' in traindata[i]):
        words  = traindata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)


In [43]:
for i in range(0, len(testdata)):
    words = ""
    if('reviewText' in testdata[i]):
        words  = testdata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [44]:
train_X[10]

'adapter work european foreign current perfectly buy wii state power supply come work foreign current burn worse case fry console purchase go replace later also price much better amazon nintendo site'

In [45]:
train_y = [ itr['overall'] for itr in traindata ] 
test_y =  [itr['overall'] for itr in testdata ]

In [46]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

In [47]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 75000, n_features: 50153


In [48]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

In [49]:
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 25000, n_features: 50153


In [50]:
X_train_tf.shape

(75000, 50153)

In [51]:
y_train_ =  np.array(train_y).reshape(75000,1)

In [52]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit( X_train_tf,y_train_ )


In [53]:
test_y_ =  np.array(test_y).reshape(25000,1)
test_y_.shape

(25000, 1)

In [55]:
y_pred = reg.predict(X_test_tf)
print(y_pred.shape)

(25000, 1)


In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

scores = cross_validate(reg,X_train_tf,y_train_ , cv=3, scoring=['neg_mean_squared_error'])
scores

{'fit_time': array([132.13234138, 122.43750453,  87.08104181, 183.60681915,
        150.16228843]),
 'score_time': array([0.41141009, 0.00299239, 0.00299191, 0.00299096, 0.00299239]),
 'test_neg_mean_squared_error': array([-3.38588701, -3.22934087, -3.67928298, -8.81118978, -3.77411772])}

In [57]:
from sklearn.metrics import mean_squared_error
y_pred = reg.predict(X_test_tf)
mean_squared_error(test_y_, y_pred)

3.739392893362879

In [60]:
reg = sklearn.linear_model.Ridge(0.01).fit( X_train_tf,y_train_ )
y_pred = reg.predict(X_test_tf)
mean_squared_error(test_y_, y_pred)

1.3439193546155443

## TF-IDF with stemmer: 

In [10]:
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

In [24]:

for d in data: 
    r = '' 
    if('reviewText' in d):
        r  = d['reviewText']
    r = ''.join([c for c in r.lower() if not c in punctuation])
    for w in r.split(): 
        w = stemmer.stem(w)
    d['reviewText'] = ''.join(w)
data_ = data 

In [26]:
data_[:2]

[{'overall': 2.0,
  'verified': True,
  'reviewTime': '11 30, 2017',
  'reviewerID': 'A2HCWBUZ5EV1ZE',
  'asin': 'B01F3Y79TW',
  'style': {'Color:': ' SA920 White Black'},
  'reviewerName': 'Cion',
  'reviewText': 't',
  'summary': 'Maximum sound level is too low.',
  'unixReviewTime': 1512000000},
 {'overall': 5.0,
  'vote': '4',
  'verified': False,
  'reviewTime': '02 26, 2014',
  'reviewerID': 'A2MUNUJDYLW0QN',
  'asin': 'B00DC7O77A',
  'style': {'Format:': ' Video Game'},
  'reviewerName': 'Boss',
  'reviewText': 'e',
  'summary': 'Very interesting game!!!',
  'unixReviewTime': 1393372800}]

In [13]:
traindata = data_[:75000]
testdata = data_[75000:100000]

In [14]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [15]:
train_X , test_X = [] , [] 

for i in range(0, len(traindata)):
    words = ""
    if('reviewText' in traindata[i]):
        words  = traindata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)


In [16]:
for i in range(0, len(testdata)):
    words = ""
    if('reviewText' in testdata[i]):
        words  = testdata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [17]:
train_y = [ itr['overall'] for itr in traindata ] 
test_y =  [itr['overall'] for itr in testdata ]

In [19]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

In [ ]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

In [ ]:
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

In [ ]:
y_train_ =  np.array(train_y).reshape(75000,1)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit( X_train_tf,y_train_ )


In [ ]:
test_y_ =  np.array(test_y).reshape(25000,1)
test_y_.shape

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = reg.predict(X_test_tf)
mean_squared_error(test_y, y_pred)

## Latent Factor 

In [47]:
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

In [48]:
algo = {}

userIDs = {}
itemIDs = {}
interactions = []

for d in traindata:
    u = d['reviewerID']
    i = d['asin']
    r = int(d['overall'])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)    
    interactions.append({
        'user_id': userIDs[u],
        'recipe_id': itemIDs[i],
        'rating': r
    })

In [49]:
userIDs = {}
itemIDs = {}
interactions_ = []

for d in testdata:
    u = d['reviewerID']
    i = d['asin']
    r = int(d['overall'])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)    
    interactions_.append({
        'user_id': userIDs[u],
        'recipe_id': itemIDs[i],
        'rating': r
    })

In [50]:
# Creation of the dataframe. Column names are irrelevant.

df = pd.DataFrame(interactions)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['user_id', 'recipe_id', 'rating']], reader)

# The columns must correspond to user id, item id and ratings (in that order).
trainset, testset = train_test_split(data, test_size=.2)

In [51]:
algo['SVD'] = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
# algo['SVD'].fit(trainset)
algo['SVD'].fit(data.build_full_trainset())
predictions = algo['SVD'].test(testset)

# Compute MSE
accuracy.mse(predictions)

MSE: 0.8469


0.8468986886773064

In [76]:
df_ = pd.DataFrame(interactions_)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

data_ = Dataset.load_from_df(df_[['user_id', 'recipe_id', 'rating']], reader)
trainset, testset = train_test_split(data_, test_size=0.85)


predictions = algo['SVD'].test(testset)

# Compute MSE
accuracy.mse(predictions)

MSE: 2.1011


2.1011406879538446

In [77]:
print(testset)

[(612, 577, 4.0), (16635, 3086, 5.0), (22538, 3318, 5.0), (21288, 7671, 4.0), (2132, 121, 4.0), (14664, 3223, 4.0), (5891, 4111, 3.0), (17627, 397, 5.0), (23402, 2215, 2.0), (4210, 1377, 5.0), (12431, 7110, 5.0), (4146, 3097, 1.0), (7109, 4751, 5.0), (6195, 2458, 4.0), (8977, 5635, 1.0), (23917, 3096, 5.0), (6134, 659, 5.0), (7514, 4960, 4.0), (19000, 3140, 5.0), (14925, 8008, 1.0), (16459, 1578, 5.0), (7783, 133, 5.0), (5249, 1359, 5.0), (23573, 123, 5.0), (7080, 4737, 1.0), (5048, 3375, 1.0), (2714, 2175, 5.0), (7657, 519, 5.0), (11932, 182, 5.0), (19535, 9559, 4.0), (12840, 4704, 1.0), (7335, 4871, 5.0), (10527, 6327, 5.0), (14765, 7753, 5.0), (22330, 841, 4.0), (21108, 254, 1.0), (23634, 10777, 5.0), (10960, 6515, 5.0), (15537, 8214, 1.0), (14285, 7798, 5.0), (23124, 251, 5.0), (12239, 7039, 5.0), (7906, 968, 5.0), (20310, 51, 1.0), (19238, 1544, 5.0), (6568, 778, 5.0), (22684, 10512, 5.0), (6188, 4263, 2.0), (23900, 685, 3.0), (8080, 5234, 1.0), (4405, 3259, 5.0), (2182, 390, 5.0)